# 使用 tensorflow 进行图像数据增广
- https://blog.csdn.net/lordofrobots/article/details/77160191
- https://blog.csdn.net/medium_hao/article/details/79227056

In [138]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf  
import numpy as np 
import random
import os
import matplotlib.pyplot as plt
from PIL import Image 
from PIL import ImageEnhance

In [139]:
dataset_dir = '../data/train_b/00c7d278-bdd6-4b5d-8026-81ee033f99f9.jpg'
value = tf.gfile.FastGFile(dataset_dir, 'rb').read()
value

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\x01\x01\x02\x01\x01\x01\x02\x02\x02\x02\x02\x04\x03\x02\x02\x02\x02\x05\x04\x04\x03\x04\x06\x05\x06\x06\x06\x05\x06\x06\x06\x07\t\x08\x06\x07\t\x07\x06\x06\x08\x0b\x08\t\n\n\n\n\n\x06\x08\x0b\x0c\x0b\n\x0c\t\n\n\n\xff\xdb\x00C\x01\x02\x02\x02\x02\x02\x02\x05\x03\x03\x05\n\x07\x06\x07\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xff\xc0\x00\x11\x08\x01\xf4\x04-\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98

In [62]:
# def adjust_brightness(value):
#     ''' 调整亮度，delta<0 变暗，delta>0 变亮 '''
#     img_data = tf.image.decode_jpeg(value, channels=3) #这个三通道效果好 
#     delta = random.uniform(-0.6, 0.6)  
#     result = tf.image.adjust_brightness(img_data, delta)
#     return result

In [140]:
def random_brightness(value):
    ''' 随机亮度，可变亮，可变暗， max_delta '''
#     img_data = tf.image.decode_jpeg(value) 
    img_data = tf.image.decode_jpeg(value, channels=3) #这个三通道效果好 
    result = tf.image.random_brightness(img_data, 0.8)
    return result

In [72]:
def random_contrast(value):
    ''' 随机对比度：降低或提高 '''
#     img_data = tf.image.decode_jpeg(value) 
    img_data = tf.image.decode_jpeg(value, channels=3) #这个三通道效果好 
    result = tf.image.random_contrast(img_data, lower=0.2, upper=1.8)
    return result

In [74]:
def random_saturation(value):
    ''' 随机饱和度：降低或提高 '''
#     img_data = tf.image.decode_jpeg(value) 
    img_data = tf.image.decode_jpeg(value, channels=3) #这个三通道效果好 
    result = tf.image.random_saturation(img_data, lower=0.2, upper=1.8)
    return result

In [123]:
def random_hue(value):
    ''' 随机色相：降低或提高 '''
#     img_data = tf.image.decode_jpeg(value) 
    img_data = tf.image.decode_jpeg(value, channels=3) #这个三通道效果好 
    result = tf.image.random_hue(img_data, 0.2)
    return result

In [143]:
%%time
with tf.Session() as sess:
    for i in range(1, 6):
        result = random_saturation(value)
        output_filename = '../data/train_b_aug/tf_random_saturation%s.jpg' % i
        plt.imsave(output_filename, result.eval())  #这个速度快 

Wall time: 2.91 s


In [130]:
def per_image_standardization(img):
    ''' 将整幅图片标准化（不是归一化），加速神经网络的训练。 '''
    if img.mode == 'RGB':
        channel = 3
    num_compare = img.size[0] * img.size[1] * channel
    img_arr=np.array(img)
    img_t = (img_arr - np.mean(img_arr))/max(np.std(img_arr), 1/num_compare)
    img_max = np.max(img_t)
    print('img_max=',img_max)
    img_min = np.min(img_t)
    print('img_min=',img_min)
    delta = img_max - img_min
    img_t = (img_t - img_min)/delta
    return img_t

In [137]:
img = Image.open(dataset_dir)
img_arr=np.array(img)
img_arr

array([[[102, 118, 108],
        [ 63,  69,  65],
        [ 76,  67,  70],
        ...,
        [ 76,  81,  49],
        [ 78,  83,  51],
        [ 80,  85,  53]],

       [[108, 124, 113],
        [ 77,  84,  77],
        [ 71,  62,  63],
        ...,
        [ 74,  79,  49],
        [ 77,  82,  52],
        [ 80,  85,  53]],

       [[129, 143, 130],
        [ 79,  84,  77],
        [ 71,  62,  63],
        ...,
        [ 78,  83,  53],
        [ 81,  86,  56],
        [ 83,  88,  58]],

       ...,

       [[ 72,  72,  72],
        [ 72,  72,  72],
        [ 72,  72,  72],
        ...,
        [ 98, 100,  87],
        [ 96,  98,  85],
        [ 93,  95,  82]],

       [[ 71,  71,  71],
        [ 72,  72,  72],
        [ 72,  72,  72],
        ...,
        [ 97,  99,  86],
        [ 95,  97,  84],
        [ 92,  94,  81]],

       [[ 71,  71,  71],
        [ 72,  72,  72],
        [ 72,  72,  72],
        ...,
        [ 96,  98,  85],
        [ 97,  99,  86],
        [ 95,  97,  84]]

In [134]:
img = Image.open(dataset_dir)
img = per_image_standardization(img)
img

img_max= 3.505562655863076
img_min= -2.085310363098528


array([[[0.4       , 0.4627451 , 0.42352941],
        [0.24705882, 0.27058824, 0.25490196],
        [0.29803922, 0.2627451 , 0.2745098 ],
        ...,
        [0.29803922, 0.31764706, 0.19215686],
        [0.30588235, 0.3254902 , 0.2       ],
        [0.31372549, 0.33333333, 0.20784314]],

       [[0.42352941, 0.48627451, 0.44313725],
        [0.30196078, 0.32941176, 0.30196078],
        [0.27843137, 0.24313725, 0.24705882],
        ...,
        [0.29019608, 0.30980392, 0.19215686],
        [0.30196078, 0.32156863, 0.20392157],
        [0.31372549, 0.33333333, 0.20784314]],

       [[0.50588235, 0.56078431, 0.50980392],
        [0.30980392, 0.32941176, 0.30196078],
        [0.27843137, 0.24313725, 0.24705882],
        ...,
        [0.30588235, 0.3254902 , 0.20784314],
        [0.31764706, 0.3372549 , 0.21960784],
        [0.3254902 , 0.34509804, 0.22745098]],

       ...,

       [[0.28235294, 0.28235294, 0.28235294],
        [0.28235294, 0.28235294, 0.28235294],
        [0.28235294, 0

In [135]:
plt.imsave(output_filename, img)  #这个速度快 
output_filename

'../data/train_b_aug/random_hue5.jpg'

In [ ]:
def distort_color(image, color_ordering=0):  
    if color_ordering == 0:  
        image = tf.image.random_brightness(image, max_delta=32. / 255.)#亮度  
        image = tf.image.random_saturation(image, lower=0.5, upper=1.5)#饱和度  
        image = tf.image.random_hue(image, max_delta=0.2)#色相  
        image = tf.image.random_contrast(image, lower=0.5, upper=1.5)#对比度  
    if color_ordering == 1:  
        image = tf.image.random_saturation(image, lower=0.5, upper=1.5)  
        image = tf.image.random_hue(image, max_delta=0.2)  
        image = tf.image.random_contrast(image, lower=0.5, upper=1.5)  
        image = tf.image.random_brightness(image, max_delta=32. / 255.)  
    if color_ordering == 2:  
        image = tf.image.random_hue(image, max_delta=0.2)  
        image = tf.image.random_contrast(image, lower=0.5, upper=1.5)  
        image = tf.image.random_brightness(image, max_delta=32. / 255.)  
        image = tf.image.random_saturation(image, lower=0.5, upper=1.5)  
    if color_ordering == 3:  
        image = tf.image.random_contrast(image, lower=0.5, upper=1.5)  
        image = tf.image.random_brightness(image, max_delta=32. / 255.)  
        image = tf.image.random_saturation(image, lower=0.5, upper=1.5)  
        image = tf.image.random_hue(image, max_delta=0.2)  
    return tf.clip_by_value(image, 0.0, 1.0)  


def preprocess_for_train(image, height, width, bbox):  
    if bbox is None:  
        bbox = tf.constant([0.0, 0.0, 1.0, 1.0], dtype=tf.float32, shape=[1, 1, 4])  
    if image.dtype != tf.float32:  
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)  
    
    # bbox_begin, bbox_size, _ = tf.image.sample_distorted_bounding_box(tf.shape(image), bounding_boxes=bbox, min_object_covered=0.1)  
    # distorted_image = tf.slice(image, bbox_begin, bbox_size)  
    # distorted_image = tf.image.resize_images(distorted_image, [height, width], method=np.random.randint(4))  

    distorted_image = tf.image.resize_images(image, [height, width], method=np.random.randint(4))  
    distorted_image = tf.image.random_flip_left_right(distorted_image)  
    distorted_image = distort_color(distorted_image, np.random.randint(4))  
    return distorted_image  


def _get_filenames_and_classes(dataset_dir):
    """Returns a list of filenames
    """
    flower_root = os.path.join(dataset_dir, 'train')
    augment_root = os.path.join(dataset_dir, 'train_augment')
    if not tf.gfile.Exists(augment_root):
        tf.gfile.MakeDirs(augment_root)
    
    photo_filenames = []
    for filename in os.listdir(flower_root): #根目录
        path = os.path.join(flower_root, filename)
        # print('path =', path)
        if os.path.isdir(path): #子目录
            augment_path = os.path.join(augment_root, filename)
        if not tf.gfile.Exists(augment_path):
            tf.gfile.MakeDirs(augment_path)
        for filename in os.listdir(path): #遍历子目录 
            fpath = os.path.join(path, filename)
            photo_filenames.append(fpath)
    return photo_filenames


def preprocess_each(filenames):
    boxes = tf.constant([[[0.0, 0.0, 1.0, 1.0], [0.1, 0.1, 0.9, 0.9]]])
    fnum = 0
    for fname in filenames:
        if fnum % 10 == 0:
            print(fnum, fname)
        fnum = fnum + 1
        value = tf.gfile.FastGFile(fname, 'rb').read()
        image_data = tf.image.decode_jpeg(value, channels=3) 
        #对一个图片进行多次增广 1~9 共 9 次，加上原图，训练集总共扩大到原来的 10 倍
        for i in range(1, 2): 
            result = preprocess_for_train(image_data, 299, 299, boxes)
            yield (i, fname, result)


def get_outputname(filename, ix):
    f_dir = os.path.dirname(filename)
#     print(f_dir)
    dir_name = os.path.basename(f_dir)
#     print(dir_name)
    p_dir = os.path.dirname(os.path.dirname(f_dir))
#     print(p_dir)
    basename = os.path.basename(filename)
#     print(basename)
    output_filename = '%s/train_augment/%s/augment%d_%s' % (p_dir, dir_name, ix, basename)
#     print(output_filename)
    return output_filename


print('dataset_dir =', FLAGS.dataset_dir)
#读取图像可任意大小  
filenames = _get_filenames_and_classes(FLAGS.dataset_dir)
# global_step = 0
step_size = 1000 #每次 1000 个 
step = 1 #共 40 次 

with tf.Session() as sess:  
#     coord = tf.train.Coordinator()  
#     threads = tf.train.start_queue_runners(coord=coord)  
  
    init = tf.global_variables_initializer()  
    sess.run(init)  
    
    for ix, fname, result in preprocess_each(filenames[(step-1)*step_size:step*step_size+1]): 
        output_filename = get_outputname(fname, ix)
#         global_step = global_step + 1
#         if global_step % 9 == 0:
#             print(global_step, output_filename)
        reimg = result.eval()  
        plt.imsave(output_filename, reimg)  #这个速度快 
  
#     coord.request_stop()  
#     coord.join(threads)  

